<a href="https://colab.research.google.com/github/lidar532/JSON2C/blob/main/JSON2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert JSON string to a "C" sprintf formatting string.
Enter or paste a JSON string below that you would like to convert to a "C" sprintf formatting statement for use in an Arduino or other project.  The JSON will be converted to a compilable "C" language formatting string you can copy to your Arduino code.  You can use [JASONata](https://try.jsonata.org/) or [JSON Editor Online](https://jsoneditoronline.org/) or other JSON tool to build, verify, and test your JSON string. Simply use "C" formatting specifiers where your values go, and JSON2C will generate the "C" formatting string for you.  Note that you can also enter a "C" comment by using the `"comment_"` JSON key and placing your comment text as the JSON value string.  You can name your "C" format variable with the JSON key `"format_name_"` with the desired "C" name in the value string.  See the example below.  I enclosed my JSON in triple quotes `'''` but it should work ok with single quotes as well.  After you enter your JSON string, press the Run arrow button to load it into the js variable use for conversion.  Additional examples are in this notebook [<img src=https://colab.research.google.com/assets/colab-badge.svg>](https://colab.research.google.com/github/lidar532/JSON2C/blob/main/JSON2C_examples.ipynb) and can be compiled and run right in Colab.

In [ ]:
#@title Example 1. Put your JSON here in the `js` variable. { form-width: "25%", display-mode: "both" }
#@markdown Run this cell , and then run Step 2 below to convert the JSON to a "C" format string.

js = '''
{ 
  "format_name_":"my_name",
  "myJSON": {
    "dest": "%s",
    "sss": "%0.2f",
    "flash": [ "%f", "%4.2lf", "%ld", "%lu", "%u", "%d", "%c" ]
  }
}
'''


In [ ]:
#@title Step 2. Click the Run button to the left to do the conversion.  { display-mode: "form" }
#@markdown Your orginal JSON will be encapsulated in 
#@markdown within a "C" ```/* comment */``` block and
#@markdown the "C" format string below.  Copy and paste the resulting 
#@markdown comments and code into your project.
import re
import os

def js2c( js, comment="" ):
  '''
  j2sc( js).  Converts a JSON message string that contains "C" language
  format specifiers into an actual "C" formatting string for use with
  sprintf or printf.  The purpose is to allow one to build and test
  a JSON string using JSONata, or other JSON tool, and then automatically
  convert it to a "C" formatting string.

  Regex expressions developed using: https://regex101.com/
  Testing JSON strings was done with: https://try.jsonata.org/
  "C" verification was done using: https://www.onlinegdb.com/#

  '''
# Extract the variable definition name, if any,
# leaving it in var_def, and the updated js with the
# var_def removed in js2

  regex = "(^\s+\"comment_\"\s*:\s*)(\")([\sa-zA-Z\.\d%!@#$%^&\*\(\)\[\]\{\}[<\>;:|,\?]+)(\")\s*,"
  m = re.search(regex, js, re.MULTILINE )
  #print(m)
  if m:
    js1 = js[0:m.start()]+js[m.end():]
    comment = f'{m.groups()[2]}'
    js1 = js[0:m.start()]+js[m.end():]
    #print(f'/*\n {comment}\n/*\n {js1}')
  else:
    comment = ""
    js1 = js

  regex = "[ \t]*\"format_name_\"\ *: *(\")(.*)(\"),[\s\n]*"
  m = re.search(regex, js1, re.MULTILINE )
  if m:
    js2 = js1[0:m.start()]+js1[m.end():]
    var_def = f'const char {m.groups()[1]}[] = ""'
  else:
    var_def = f'const char var_def[] = ""'
    js2 = js1
  #return js2

  regex = r"(\")([%][\d]*\.*[\d]*l?f)(\")"          # %f %lf
  subst = "\\2"
  js3 = re.sub(regex, subst, js2, 0, re.MULTILINE)
  #return js4

  regex = r"(\")([%][\d]*[ul]?[ldu])(\")"           # %u %d %ul %ld
  subst = "\\2"
  js4 = re.sub(regex, subst, js3, 0, re.MULTILINE)
  #return js4

  regex = r"(\")([%][\d]*[sc])(\")"                 # %c, %s or %10s
  subst = "~~~\\2~~~"                               # mark strings with ~~~
  js5 = re.sub(regex, subst, js4, 0, re.MULTILINE)
  #return js5

  js6 = js5.replace('"','\\"')                      # Now escape all quotes
  #return js6

  js7 = js6.replace('~~~','\\"')                    # Now, repace the ~~~ with escaled quotes
  #return js7

  regex = r"(\n\s*\n)"                              # remove blank lines
  subst = ""
  js8 = re.sub(regex, subst, js7, 0, re.MULTILINE)
  #return js8

  regex = r"(  +)"                                  # remove excess spaces.
  subst = ""
  js9 = re.sub(regex, subst, js8, 0, re.MULTILINE)

  js10 = js9.replace('\n', '\"\n\"')                 # Put quotes at the beg and end of all lines.
  #return js10

  js11 = f'"{js10}"'                                 # add to beg and end of the string
  #return js11

  js12 = f'{var_def}\n{js11};'
  #return js12

  t = js12.splitlines(keepends=True)                # indent "C" vars.
  v = "    ".join(t[1:])                               # Add tabs  
  js12a = t[0]+v                                     # prepend first line                                     
  #return js12a

  doc = "/* This was autogenerated from the above JSON using js2c.ipynb */"
  js13 = f'/* {comment} {js} */\n\n{doc}\n{js12a}\n\n'
  return js13

if 'js' in globals():
  jsc = js2c(js)
  print(jsc)
else:
  print('You need to put some JSON in variable `js` first.')

## Additional Example JSON messages.  
The cells below each have more examples of JSON messages.  To use them
simple run the example cell, and then run "Step 2." above to convert 
the JSON message into a "C" format string.  You can paste the example 
JSON into https://try.jsonata.org/ to verify it.


In [ ]:
#@title Example 2. { form-width: "25%", display-mode: "both" }
#@markdown Run this cell , and then run Step 2 below to convert the JSON to a "C" format string.

js = '''
  {
     "sensor": "%s",
     "time": %d,
     "data": [ %f, %f]
   }

'''

In [ ]:
#@title Example 3. GeoJSON Example. { form-width: "25%", display-mode: "both" }
#@markdown see: https://geojson.org/
#@markdown Run this cell , and then run Step 2 below to convert the JSON to a "C" format string.

js = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": ["%f", "%f"]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}
'''

In [ ]:
#@title Example 4. Some nested JSON with GPS data. { form-width: "25%", display-mode: "both" }
js = '''
  {
    "format_name_" : "JSON_gps_status_fmt",
     "ppksyslogger": {
       "sss" : "%f",
       "gps_stat" : {
         "utc" : "%9.2f",
         "nsat" : "%d",
         "Q" : "%d",
         "GPS_good" : "%d",
         "raw_b" : "[%ld,%ld]",
         "ascii_b" : "[%ld,%ld]",
         "pass_b" : "[%ld,%ld]"
       }
     }
   } 
'''

This notebook lives at: https://github.com/lidar532/JSON2C.